# Preprocessing Data - Fundamental Techniques
<br>Just I want to bring up some techniques of data preparation before putting data to a model. Most of the machine learning algorithm will have better performance if data be prepared and cleaned before training. 

In [32]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from datetime import date 
url = 'https://www.stats.govt.nz/assets/Uploads/Electronic-card-transactions/Electronic-card-transactions-March-2020/Download-data/electronic-card-transactions-march-2020-csv.zip'
df = pd.read_csv(url)

## 1- Imputation
<br>First method is null dropping based on a threshold. For instance if more than (75% = threshold) of a column or row would be null, they will be removed.

In [ ]:
threshold = 0.75
#first remove fetures that has many null value (more than threshold)
df = df[df.columns[df.isnull().mean() <= threshold]]

#second remove rows that has many null value (more than threshold)
df = df.loc[df.isnull().mean(axis=1) <= threshold]

What if we have data with many nulls and we don't want to remove useful information from dataframe,
replacement seems to be a good idea. We can replace the nulls with many things like 0 or ...
For instance if we have a column that has just two values like 1 and NanN, zero is a good replacement. or 
if we have a columns like "customer visit count in last month" and we have many Nan, zero is a good replacement.<br>
we can also use median of a column to replacement (not average because average is sensitive to outliers)

In [ ]:
#relpace with median
#df.fillna(0)
#df['column name'].fillna(something)

#here we replace nulls with median
df['Data_value'].fillna(df['Data_value'].median())

#here we replace nulls with the most ferequency value in that column 
#suitable for categorical data replacement 
df['Series_title_3'].fillna(df['Series_title_3'].value_counts().idxmax())

## 2- Handling outliers
<br>There are many ways to find the outliers. I believe the best way is to see your data with plotting each column. All other methods are open to making mistakes.
Here I'm trying to find outliers with:
<br>1-Upper and lower limit with standard deviation and mean
<br>2-Z-Score
<br>3-Upper and lower limit with percentile
<br>Then we show that how we can drop or cap those outliers


In [4]:
# Finding outlier using upper and lower limit with standard deviation and mean

#define a facotor between 2 and 4
factor = 3
upper_limit = df['Data_value'].mean() + df['Data_value'].std()*3
lower_limit = df['Data_value'].mean() - df['Data_value'].std()*3

#remove outliers from data
df = df.loc[df['Data_value'] <= upper_limit & df['Data_value'] >= lower_limit]

#cap
df.loc[df['Data_value'] > upper_limit, 'Data_value'] = upper_limit
df.loc[df['Data_value'] < lower_limit, 'Data_value'] = lower_limit

Z-score can quantify the unusualness of an observation when your data follow the normal distribution z-scores are the numbers of standard deviation above and below the mean that each value falls. For example a z-score of 2 signifies it is two standard deviations above the mean while a z-score of -2 signifies it is two standard deviation below the mean. A z-score of zero represents a value that equals the mean.
<br>Z= (X-M)/Sigma<br>
The further away an observations z-score is from zero, the more unusual it is.

In [16]:
# Create a full number dataframe (choosing the numeric columns just for test)
df_test = df[['Period','Data_value','Magnitude']]

# finding outlier using Z-Score
scaler = preprocessing.StandardScaler()
col_list = df_test.columns
scaled_df_test = scaler.fit_transform(df_test)
df_test = pd.DataFrame(scaled_df_test,columns = col_list)

I can assume a certain percent of the value from the top or the bottom as an outlier. The key point is here to set the percentage value, and this depends on the distribution of data. If data ranges from 0 to 100, top 5% is not the values between 96 and 100. Top 5% means here the values that are out of the 95th percentile of data.

In [34]:
upper_limit = df['Data_value'].quantile(0.95)
lower_limit = df['Data_value'].quantile(0.05)

#drop
df = df.loc[(df['Data_value'] <= upper_limit) & (df['Data_value'] >= lower_limit)]

#cap
df.loc[(df['Data_value'] > upper_limit),'Data_value'] = upper_limit
df.loc[(df['Data_value'] < lower_limit),'Data_value'] = lower_limit

## 3- Binning 
<br>Can be applied on both categorical and numerical data. The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time we bin something, we sacrifies information and make data more regularized. The trade of between performance and overfitting is the keypoint of binning the process.<br>
for example, we have values between 0 to 100 and we need to bin them,<br>
0  to 30  ---> low<br>
31 to 70  ---> medium<br>
71 to 100 ---> high<br>

or in categorical data<br>

Iran      ---> Asia<br>
France    ---> Europe<br>
Argantina ---> South America<br>
China     ---> Asia<br>
Spain     ---> Europe<br>
Brazil    ---> South America<br>
India     ---> Asia<br>
Italy     ---> Europe<br>

In [42]:
#Numerical binning
df['Data_value_bin'] = pd.cut(df['Data_value'], bins= [1,100000,1000000,10000000,100000000], labels=['a','b','c','d'])

#Categorical binning
conditions = [df['country'].str.contains('Spain'),
              df['country'].str.contains('Iran'),
              df['country'].str.contains('France'),
              df['country'].str.contains('Argantina')]
choiselist = ['Eroupe','Asia','Eroup','South America']
df['continent'] = np.select(conditions,choiselist,default='other')

## 4- Log Transform
<br>Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations in feature engineering. What are the benefits of log transform:<br>
<br>1- It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.<br>
<br>2- In most of the cases the magnitude order of the data changes within the range of the data. For instance, the difference between ages 15 and 20 is not equal to the ages 65 and 70. In terms of years, yes, they are identical, but for all other aspects, 5 years of difference in young ages mean a higher magnitude difference. This type of data comes from a multiplicative process and log transform normalizes the magnitude differences like that.<br>
<br>3- It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

In [ ]:
df['log+1'] = (df['col']+1).transform(np.log)

#if we have negetive numbers in col, we can handle it like this

df['log+1'] = (df['col'] - df['col'].min() + 1).transform(np.log)

## 5- One Hot Encoder (OHE)
<br>For injecting categorical data to our models we need to encode them while the numbers shouldn't relate to each other. For example if we have colors column with blue red and green values we cannot map them to 1,2,3 becuase  3 > 2 but green is not greater than red. 

In [ ]:
# we need list of categorical columns in dataframe
cat_list = ['color','car']

#call get_dummies from pandas and put the categorical data into it
encoded_cat_list = np.get_dummies(df[cat_list])

#add encoded dataframe and drop cat_list
df.join(encoded_cat_list).drop(cat_list,axis=1)

## 6- Grouping Operations
<br>In most machine learning algorithms, every instance is represented by a row in the training dataset, where every column show a different feature of the instance. This kind of data called “Tidy”.<br><br>
Datasets such as transactions rarely fit the definition of tidy data above, because of the multiple rows of an instance. In such a case, we group the data by the instances and then every instance is represented by only one row.<br><br>
The key point of group by operations is to decide the aggregation functions of the features. For numerical features, average and sum functions are usually convenient options, whereas for categorical features it more complicated.<br><br>
The first option is to select the label with the highest frequency. In other words, this is the max operation for categorical columns, but ordinary max functions generally do not return this value, you need to use a lambda function for this purpose.<br>
df.groupby('field we want to group by').agg(aggregate function)<br><br>
Second option is to make a pivot table.

In [60]:
# we create a data-frame based for option #1
mylist = [[1,'Moe'],[1,'Moe'],[1,'three'],[2,'Moe'],[3,'Moe']]
col_list = ['id','name']
df = pd.DataFrame(mylist,columns = col_list )
df.head()

,id,name
0,1,Moe
1,1,Moe
2,1,three
3,2,Moe
4,3,Moe


In [59]:
df.groupby('id').agg(lambda x: x.value_counts().idxmax())

,name
id,
1,Moe
2,Moe
3,Moe


In [62]:
# we create a data-frame based for option #2
mylist = [[1,'Roma',1],[2,'Madrid',2],[1,'Madrid',1],[3,'Istanbul',1],[2,'Istanbul',4],[1,'Istanbul',3],[1,'Roma',3]]
col_list = ['user','city','visit_days']
df = pd.DataFrame(mylist,columns = col_list)

In [63]:
df.head()

,user,city,visit_days
0,1,Roma,1
1,2,Madrid,2
2,1,Madrid,1
3,3,Istanbul,1
4,2,Istanbul,4


In [65]:
# pivot
df.pivot_table(index = 'user', columns = 'city' , values = 'visit_days' , aggfunc = np.sum , fill_value =0)

city,Istanbul,Madrid,Roma
user,,,
1,3,1,4
2,4,2,0
3,1,0,0


## 7- Splitting Featuers<br>
Most of the time the dataset contains string columns that violate tidy data principles. By extracting the utilizable par of a column into new features, <br> 1-Enabling ML algorithms to comprehend them<br> 2-make possible to bin them<br> 3-improve model performance by uncovering potential information<br>
Split function is a good option, however there is no one way of splitting features. It depends on the characteristics of the column, how to split it.

In [71]:
my_list = [['Moe Salari'],['Maryam Salari'],['Esmat Gonabadi'],['Mahmoud Salari']]
col_name = ['fullname']
df = pd.DataFrame(my_list, columns=col_name)

In [73]:
df['first name'] = df['fullname'].str.split(' ').map(lambda x: x[0])
df['last name'] = df['fullname'].str.split(' ').map(lambda x: x[-1])

In [74]:
df

,fullname,first name,last name
0,Moe Salari,Moe,Salari
1,Maryam Salari,Maryam,Salari
2,Esmat Gonabadi,Esmat,Gonabadi
3,Mahmoud Salari,Mahmoud,Salari


## 8- Scaling 
Normalization: (Xi - Xmin)/(Xmax - Xmin)<br>
Standardization: (X - Mean) / SD

In [80]:
#Suppose we have a data-frame like this,
mylist = [[10,20,30],[1222,134,8734],[32,32984,98732],[9823,223,93],[321,3214,456],[132,45,123]]
col_list = ['col1','col2','col3']
df = pd.DataFrame(mylist,columns = col_list)

In [76]:
df

,col1,col2,col3
0,10,20,30
1,1222,134,8734
2,32,32984,98732
3,9823,223,93
4,321,3214,456
5,132,45,123


In [77]:
# Normalization
X = df.values #fetch list
MMS = preprocessing.MinMaxScaler() #create object
df_scaled = MMS.fit_transform(df) # transform 
df = pd.DataFrame(df_scaled,columns = col_list) #dataframe

In [78]:
df

,col1,col2,col3
0,0.000000,0.000000,0.000000
1,0.123510,0.003458,0.088185
2,0.002242,1.000000,1.000000
3,1.000000,0.006158,0.000638
4,0.031693,0.096894,0.004316
5,0.012432,0.000758,0.000942


In [82]:
#Standardization 
X = df.values
col_list = df.columns
SS = preprocessing.StandardScaler()
scaled_X = SS.fit_transform(X)
df = pd.DataFrame(scaled_X, columns = col_list)

In [83]:
df

,col1,col2,col3
0,-0.537916,-0.503794,-0.496809
1,-0.197173,-0.494353,-0.256547
2,-0.531730,2.226134,2.227717
3,2.220916,-0.486982,-0.495070
4,-0.450481,-0.239281,-0.485050
5,-0.503616,-0.501724,-0.494242


## 9- Extracting Date
<br>Building an ordinal relationship between the values is very challenging for ML algorithms if you leave a data column without manipulating.<br>
1- Extract the part of the date into different columns, year, month, day, etc.<br>
2- Extracting the period between the current date and columns in terms of years, month, day, etc.<br>
3- Extracting some specific features from the date. Name of the weekday, weekend, holyday, etc

In [93]:
#prepare a dataframe
myList = [['19840522'],['20170102'],['20200202'],['20000101']]
col_name = ['date']
df = pd.DataFrame(myList,columns=col_name)

In [95]:
#convert string to datetime
df['date'] = pd.to_datetime(df['date'], format = '%Y%m%d')

In [97]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['passyear'] = date.today().year - df['year']

In [98]:
df

,date,year,month,day,passyear
0,1984-05-22,1984,5,22,36
1,2017-01-02,2017,1,2,3
2,2020-02-02,2020,2,2,0
3,2000-01-01,2000,1,1,20
